Firstly I import all the necessary libraries. 

In [196]:
from bs4 import BeautifulSoup
import urllib2
import numpy as np
import pandas as pd

Next step is to define the functions get_cols and get_data which given a table will return parsed out column names and the actual data respectively

In [197]:
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/1').read()
soup = BeautifulSoup(page)
table_div = soup.find(id='my-players-table')
table = table_div.find("table")

def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

col = get_cols(table)
#print col

def get_data(table):
    table_rows = table.findAll('tr')
    players_stats_array = []
    for row in table_rows:
        if row.attrs['class'][0]=='colhead':
            continue
        player_stats = []
        row_cols = row.find_all('td')
        col_vals = []
        player_col = row_cols[1]
        player_name = player_col.find('a').string
        player_position = player_col.contents[1]
        player_position = player_position.split(' ')[1]
        player_stats.append(player_name)
        player_stats.append(player_position)
        for i in range(2, len(row_cols)):
            stat = row_cols[i].string
            player_stats.append(stat)
        players_stats_array.append(player_stats)
    return players_stats_array

Then I will use get_data to switch between the 13 pages throughout which the data is split into and make sure I obtain the data from all of them

In [198]:
player_stats_scoring = []

for i in range(1,482,40):
    url = 'http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/'+ str(i)
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    table_div = soup.find(id='my-players-table')
    table = table_div.find("table")
    player_stats_i = get_data(table)
    player_stats_scoring.extend(player_stats_i)
#print player_stats_all

Converting all the scoring results to a Pandas DataFrame called df_scoring

In [199]:
np_array = np.array(player_stats_scoring)
df_scoring = pd.DataFrame(np_array,columns=col)
df_scoring.head(30)

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Russell Westbrook,PG,OKC,67,34.4,28.1,9.4-22.0,.426,1.3-4.3,.299,8.1-9.8,.835
1,James Harden,SG,HOU,81,36.8,27.4,8.0-18.1,.440,2.6-6.9,.375,8.8-10.2,.868
2,Kevin Durant,SF,OKC,27,33.8,25.4,8.8-17.3,.510,2.4-5.9,.403,5.4-6.3,.854
3,LeBron James,SF,CLE,69,36.1,25.3,9.0-18.5,.488,1.7-4.9,.354,5.4-7.7,.710
4,Anthony Davis,PF,NO,68,36.1,24.4,9.4-17.6,.535,0.0-0.2,.083,5.5-6.8,.805
5,Carmelo Anthony,SF,NY,40,35.7,24.2,9.0-20.2,.444,1.5-4.5,.341,4.7-5.9,.797
6,DeMarcus Cousins,C,SAC,59,34.1,24.1,8.4-18.1,.467,0.0-0.1,.250,7.2-9.2,.782
7,Stephen Curry,PG,GS,80,32.7,23.8,8.2-16.8,.487,3.6-8.1,.443,3.9-4.2,.914
8,LaMarcus Aldridge,PF,POR,71,35.4,23.4,9.3-19.9,.466,0.5-1.5,.352,4.3-5.1,.845
9,Kobe Bryant,SG,LAL,35,34.5,22.3,7.6-20.4,.373,1.5-5.3,.293,5.6-6.9,.813


Repeat the process in order to obtain the assists data and insert it into a separate dataframe called df_assists

In [200]:
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false').read()
soup = BeautifulSoup(page)
table_div = soup.find(id='my-players-table')
table = table_div.find("table")

def get_cols(table):
    table_head = table.find(attrs={"class":'colhead'})
    header_cols = table_head.findAll('td')
    cols = []
    for header_col in header_cols:
        val = header_col.string
        if val != 'RK':
            cols.append(val)
        if val == 'PLAYER':
            cols.append('POSITION')
    return cols

col = get_cols(table)
player_stats_assists = []

for i in range(1,482,40):
    url = 'http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/count/'+ str(i)
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page)
    table_div = soup.find(id='my-players-table')
    table = table_div.find("table")
    player_stats_i = get_data(table)
    player_stats_assists.extend(player_stats_i)
    
#print player_stats_all

np_array = np.array(player_stats_assists)
df_assists = pd.DataFrame(np_array,columns=col)
df_assists.head(30)

,PLAYER,POSITION,TEAM,GP,MPG,AST,APG,TO,TOPG,AP48M,AST/TO
0,Chris Paul,PG,LAC,82,34.8,838,10.2,190,2.3,14.1,4.41
1,John Wall,PG,WSH,79,35.9,792,10.0,304,3.8,13.4,2.61
2,Ty Lawson,PG,DEN,75,35.5,720,9.6,185,2.5,13.0,3.89
3,Ricky Rubio,PG,MIN,22,31.5,193,8.8,64,2.9,13.4,3.02
4,Russell Westbrook,PG,OKC,67,34.4,574,8.6,293,4.4,12.0,1.96
5,Rajon Rondo,PG,BOS/DAL,68,29.7,538,7.9,210,3.1,12.8,2.56
6,Stephen Curry,PG,GS,80,32.7,619,7.7,249,3.1,11.4,2.49
7,LeBron James,SF,CLE,69,36.1,511,7.4,272,3.9,9.8,1.88
8,Jeff Teague,PG,ATL,73,30.5,513,7.0,205,2.8,11.1,2.50
9,James Harden,SG,HOU,81,36.8,565,7.0,321,4.0,9.1,1.76


In [201]:
df_scoring.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Russell Westbrook,PG,OKC,67,34.4,28.1,9.4-22.0,.426,1.3-4.3,.299,8.1-9.8,.835
1,James Harden,SG,HOU,81,36.8,27.4,8.0-18.1,.440,2.6-6.9,.375,8.8-10.2,.868
2,Kevin Durant,SF,OKC,27,33.8,25.4,8.8-17.3,.510,2.4-5.9,.403,5.4-6.3,.854
3,LeBron James,SF,CLE,69,36.1,25.3,9.0-18.5,.488,1.7-4.9,.354,5.4-7.7,.710
4,Anthony Davis,PF,NO,68,36.1,24.4,9.4-17.6,.535,0.0-0.2,.083,5.5-6.8,.805


In [202]:
df_assists.head()

,PLAYER,POSITION,TEAM,GP,MPG,AST,APG,TO,TOPG,AP48M,AST/TO
0,Chris Paul,PG,LAC,82,34.8,838,10.2,190,2.3,14.1,4.41
1,John Wall,PG,WSH,79,35.9,792,10.0,304,3.8,13.4,2.61
2,Ty Lawson,PG,DEN,75,35.5,720,9.6,185,2.5,13.0,3.89
3,Ricky Rubio,PG,MIN,22,31.5,193,8.8,64,2.9,13.4,3.02
4,Russell Westbrook,PG,OKC,67,34.4,574,8.6,293,4.4,12.0,1.96


Merge the two dataframes with a left join, remove duplicate columns and rename the columns to remove the '_x' suffixes

In [203]:
df = pd.merge(df_scoring, df_assists, on='PLAYER', how='left')
df = df.drop(['POSITION_y','TEAM_y','GP_y','MPG_y'],1)
df.columns=['PLAYER','POSITION','TEAM','GP','MPG','PTS','FGM-FGA','FG%','3PM-3PA','3P%','FTM-FTA','FT%','AST','APG','TO','TOPG','AP48M','AST/TO']
df.head(500)

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%,AST,APG,TO,TOPG,AP48M,AST/TO
0,Russell Westbrook,PG,OKC,67,34.4,28.1,9.4-22.0,.426,1.3-4.3,.299,8.1-9.8,.835,574,8.6,293,4.4,12.0,1.96
1,James Harden,SG,HOU,81,36.8,27.4,8.0-18.1,.440,2.6-6.9,.375,8.8-10.2,.868,565,7.0,321,4.0,9.1,1.76
2,Kevin Durant,SF,OKC,27,33.8,25.4,8.8-17.3,.510,2.4-5.9,.403,5.4-6.3,.854,110,4.1,74,2.7,5.8,1.49
3,LeBron James,SF,CLE,69,36.1,25.3,9.0-18.5,.488,1.7-4.9,.354,5.4-7.7,.710,511,7.4,272,3.9,9.8,1.88
4,Anthony Davis,PF,NO,68,36.1,24.4,9.4-17.6,.535,0.0-0.2,.083,5.5-6.8,.805,149,2.2,95,1.4,2.9,1.57
5,Carmelo Anthony,SF,NY,40,35.7,24.2,9.0-20.2,.444,1.5-4.5,.341,4.7-5.9,.797,122,3.1,89,2.2,4.1,1.37
6,DeMarcus Cousins,C,SAC,59,34.1,24.1,8.4-18.1,.467,0.0-0.1,.250,7.2-9.2,.782,210,3.6,254,4.3,5.0,0.83
7,Stephen Curry,PG,GS,80,32.7,23.8,8.2-16.8,.487,3.6-8.1,.443,3.9-4.2,.914,619,7.7,249,3.1,11.4,2.49
8,LaMarcus Aldridge,PF,POR,71,35.4,23.4,9.3-19.9,.466,0.5-1.5,.352,4.3-5.1,.845,124,1.7,122,1.7,2.4,1.02
9,Kobe Bryant,SG,LAL,35,34.5,22.3,7.6-20.4,.373,1.5-5.3,.293,5.6-6.9,.813,197,5.6,128,3.7,7.8,1.54


Convert appropriate fields to float and integer

In [204]:
df[['MPG', 'PTS','FG%','3P%','FT%','APG','TOPG','AP48M','AST/TO']] = df[['MPG', 'PTS','FG%','3P%','FT%','APG','TOPG','AP48M','AST/TO']].astype(float)
df[['GP','AST','TO']] =df[['GP','AST','TO']].astype(int) 